# Credit Card Fraud Detection
**Description given by the data providers**

- The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

- It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

In [1]:
import numpy as np
import pandas as pd

## Importing the Data
- Data reference: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [2]:
fraud_df = pd.read_csv('creditcard.csv')

### A note about the data
- **Time** is the number of seconds elapsed between this transaction and the first transaction in the dataset
- **Amount** is the amount spent on that transaction
- **Class** is the label on whether the transaction is fradulent or not
- **V1 - V28** are anonymized vectors to protect the users who made these transactions. They are unlabelled features that have undergone a dimensionality reduction with **Principal Component Analysis**

In [6]:
fraud_df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

## Checking the Fradulent Cases
- Let's now make sure that the data does indeed have 492 fradulent transactions

In [22]:
def class_identifier(data):
    fraudCounter, notFraudCounter = 0, 0
    for i in range(len(data)):
        if data['Class'][i] == 1:
            fraudCounter += 1
        else:
            notFraudCounter += 1
    return fraudCounter, notFraudCounter

In [24]:
numberOfFrauds, numberOfClean = class_identifier(fraud_df)

### Note that it is indeed the case that only 492 cases of Fraud Exist here

This suggests an imbalance in the data. If a classification algorithm is to be built without overfitting, 

In [28]:
print(f"Number of Fraud Transactions: {numberOfFrauds}")
print(f"Number of Clean Transactions: {numberOfClean}")

Number of Fraud Transactions: 492
Number of Clean Transactions: 284315
